<a href="https://colab.research.google.com/github/utkarshut/Deep-Learning--MLP-RNN-LSTM-Projects/blob/master/cats_dogs_on_vgg16_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
#from sklearn.metrics import plot_confusion_matrix
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.models import load_model
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
model = keras.applications.vgg16.VGG16()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!unzip '/content/drive/My Drive/data.zip'

Archive:  /content/drive/My Drive/data.zip
replace data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/data/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/test/dogs/dog.991.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/data/test/dogs/._dog.991.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/test/dogs/dog.985.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [8]:
model.save('./VGG16_model.h5')
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
type(model)

keras.engine.training.Model

In [0]:
model_s = Sequential()
for layers in model.layers:
    model_s.add(layers)

In [13]:
model_s.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [11]:
model_s.layers.pop()

In [0]:
for layer in model_s.layers:
    layer.trainable=False

In [0]:
model_s.add(Dense(2,activation='softmax'))

In [14]:
model_s.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
model_s.save('./vgg16_2op.h5')

In [0]:
#model_s=load_model('./vgg16_2op.h5')

In [0]:
train_data_dir = './data/train'
valid_data_dir = './data/validation'
test_data_dir = './data/test'

In [17]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=10,
        classes=['dogs','cats'])  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        valid_data_dir,
        target_size=(224, 224),
        batch_size=10,
        classes=['dogs','cats'])   #class_mode='binary')
# this is a similar generator, for validation data
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_data_dir,
        target_size=(224, 224),
        batch_size=10,
        classes=['dogs','cats'])   #class_mode='binary')

Found 1950 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [18]:
# dimensions of our images.
img_width, img_height = 224, 224
nb_train_samples = 2000
nb_validation_samples = 1000
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
print(input_shape,'input_shape')

(224, 224, 3) input_shape


In [19]:
print(train_generator.class_indices)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

{'dogs': 0, 'cats': 1}
{0: 'dogs', 1: 'cats'}


In [20]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
print(STEP_SIZE_TRAIN,STEP_SIZE_VALID,STEP_SIZE_TEST)

195 100 5


In [0]:
model_s.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [24]:
model_s.fit_generator(generator=train_generator,validation_data=valid_generator,\
                    steps_per_epoch=STEP_SIZE_TRAIN,validation_steps=STEP_SIZE_VALID,verbose=1,epochs=30)

Epoch 1/30
195/195 [==============================] - 13s 64ms/step - loss: 0.6269 - acc: 0.7226 - val_loss: 0.6199 - val_acc: 0.7410
Epoch 2/30
195/195 [==============================] - 11s 58ms/step - loss: 0.6253 - acc: 0.7138 - val_loss: 0.6183 - val_acc: 0.7410
Epoch 3/30
195/195 [==============================] - 12s 61ms/step - loss: 0.6238 - acc: 0.7226 - val_loss: 0.6167 - val_acc: 0.7390
Epoch 4/30
195/195 [==============================] - 12s 59ms/step - loss: 0.6224 - acc: 0.7215 - val_loss: 0.6152 - val_acc: 0.7400
Epoch 5/30
195/195 [==============================] - 12s 59ms/step - loss: 0.6210 - acc: 0.7241 - val_loss: 0.6137 - val_acc: 0.7400
Epoch 6/30
195/195 [==============================] - 11s 58ms/step - loss: 0.6195 - acc: 0.7226 - val_loss: 0.6122 - val_acc: 0.7410
Epoch 7/30
195/195 [==============================] - 12s 61ms/step - loss: 0.6182 - acc: 0.7231 - val_loss: 0.6107 - val_acc: 0.7410
Epoch 8/30
195/195 [==============================] - 11s 58ms